# Using LangChain with GPT4All and Custom Prompts

## References

1. [LangChain docs on using the GPT4All library](https://python.langchain.com/docs/integrations/llms/gpt4all)

## Code

### Import libraries

In [9]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

### Setup

In [11]:
local_path = r"C:\Users\deshi\Code\gpt4all-models\orca-mini-3b-gguf2-q4_0.gguf"

In [12]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

### Standard template

In [10]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [13]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

### Inference

In [16]:
question = """
Write a program to calculate the nth fibonacci number given a user input integer n. Also, print out all the n fibonacci numbers leading upto n in a list.
"""

llm_chain.run(question)